# The Hand class

This notebook provides some documentation about Hand objects.

## Importing the Hand class

The current working directory needs to be "fivecarddraw" in order to run the code cells of this notebook. Assuming the fivecarddraw repository has been cloned, then this notebook is contained within the "fivecarddraw/notebooks" folder. To change the current working directory, run the next cell.

In [1]:
from os import chdir, getcwd

if not getcwd().endswith("fivecarddraw"):
    chdir("..")
    
print(f"Current Directory: {getcwd()}")

Current Directory: d:\My Projects\Python\Games\fivecarddraw


Then the Hand class can be imported in the usual pythonic way.

In [2]:
from fivecarddraw import Hand

## Creating a Hand

A Hand is inherently a list of Card objects. This gives it access to list methods and the capability to be indexed. Unlike a Deck object, which always contains 52 specific cards, a Hand object requires a collection of Cards as an argument. Obtaining these from a Deck ensures the hand can not contain multiple copies of the same card.

In [3]:
from fivecarddraw import Deck

deck = Deck()

In [280]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

Hand: 10♣ 8♡ Q♠ A♡ 6♣


## Evaluating the Strength of a Hand

The role of hand evaluation has been given to Dealer objects, to makes use of large memos, that speed up the time required to evaluate the hand. More details about the Dealer object can be found in the dealer.ipynb notebook. The technique used to evaluate hands is based upon "Cactus Kev's Poker Hand Evaluator". Details of how it works can be found here: http://suffe.cool/poker/evaluator.html.

### Hand Evaluation Memos

#### Lookup-Tables

The memos FLUSH_RANKS, UNIQUE_FIVE_RANKS and DUPE_RANKS pull data from lookup-tables. The lookup-tables enumerate all possible card combinations which make up a hand, and provide labels of the strength, both quantitative and qualitative. The lookup-tables were scraped from: http://suffe.cool/poker/7462.html.

In [18]:
handle = open("lookup-tables/flush lookup.txt", "r")
print(handle.readline())
handle.close()

SF  AKQJT  1



The first two letters provided a qualitative label of the hand strength. The next five letters give the combination of cards. The remaining number gives a value from 1 to 7462, providing a quantitative label for the hand strength; 1 is the strongest and 7462 is the weakest. In the example above, "SF" refers to straight flush; "AKQJT" refers to ace-king-queen-jack-ten in any order, with identical suits (implied by the qualitative label "flush"), and "1" refers to an ace-king-queen-jack-ten straight flush being the number 1 strongest hand (a royal flush).

#### Unique Keys for the Memos

Using the lookup-tables,the memos FLUSH_RANKS, UNIQUE_FIVE_RANKS and DUPE_RANKS are populated. They encode card combinations and quantitative strengths as key-value pairs of a dictionary. The card combination encodings are derived from the representations of cards as 32bit integers, mentioned in the card.ipynb notebook.

All possible card values are mapped to prime numbers, and all possible card values are mapped to a 13bit mask. This allows for unique keys to be created for any combination of cards in a hand, suitable for whichever memo it is required for.  

In [24]:
VALUES = {char : 2 ** i for i, char in enumerate("23456789TJQKA")}
PRIMES = {char : p for char, p in zip("23456789TJQKA", Deck()[0].PRIMES)}

How these are used explicity is covered in the next 3 subsections.

#### Checking for Flushes, Straight Flushes or Royal Flushes

The keys of the FLUSH_RANKS memo are populated by the values of cards in each combination and the bitwise OR operation, to give a 13bit encoding of every possible flush hand. The keys are mapped to a number between 1 to 7462 corresponding to the strength of the card combination that the key represents. All combinations and strengths are taken from flush_lookup.txt.

In [25]:
FLUSH_RANKS = {
    VALUES[line[4]] | VALUES[line[5]] | 
    VALUES[line[6]] | VALUES[line[7]] | 
    VALUES[line[8]] : int(str(line)[11:]) 
    for line in open("lookup-tables/flush lookup.txt", "r")}

Before a hand is used as a key for FLUSH_RANKS to obtain its strength, it should be checked to see if it contains a flush. This is because not every hand is included in FLUSH_RANKS. Checking the hand contains a flush can be done using the bitwise AND operation, the 32bit card integers, and a bitwise mask to extract suit information. If a hand has a flush, then the 32 bit integer of each card will share an integer representing a suit.

In [27]:
def CheckFlush(hand):
    suit_mask = (15 << 12)
    return hand[0].b & hand[1].b & hand[2].b & hand[3].b & hand[4].b & suit_mask

In [89]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

print(f"{hand[0]}   Suit: {format(((hand[0].b >> 12) & 15), '#06b')[2:]} & {hand[0].MASK[-16:-12]}")
print(f"{hand[1]}   Suit: {format(((hand[1].b >> 12) & 15), '#06b')[2:]} & {hand[1].MASK[-16:-12]}")
print(f"{hand[2]}   Suit: {format(((hand[2].b >> 12) & 15), '#06b')[2:]} & {hand[2].MASK[-16:-12]}")
print(f"{hand[3]}   Suit: {format(((hand[3].b >> 12) & 15), '#06b')[2:]} & {hand[3].MASK[-16:-12]}")
print(f"{hand[4]}   Suit: {format(((hand[4].b >> 12) & 15), '#06b')[2:]} & {hand[4].MASK[-16:-12]}")
print(f"Combination: {format((CheckFlush(hand) >> 12), '#06b')[2:]}")

Hand: J♣ 5♠ 9♡ A♢ K♢
J♣   Suit: 1000 & ♣♢♡♠
5♠   Suit: 0001 & ♣♢♡♠
9♡   Suit: 0010 & ♣♢♡♠
A♢   Suit: 0100 & ♣♢♡♠
K♢   Suit: 0100 & ♣♢♡♠
Combination: 0000


The encoding of a card combination, to be used as a key for FLUSH_RANKs, can be extracted with the bitwise OR operation, the 32bit card integers and bitshifting. If the hand contains five cards with different values, then the encoding will have 5 bits switched on.

In [73]:
def GetEncoding(hand):
    return (hand[0].b | hand[1].b | hand[2].b | hand[3].b | hand[4].b) >> 16

In [281]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

print(f"{hand[0]}   Value: {format((hand[0].b >> 16), '#015b')[2:]} & {hand[0].MASK[-29:-16]}")
print(f"{hand[1]}   Value: {format((hand[1].b >> 16), '#015b')[2:]} & {hand[1].MASK[-29:-16]}")
print(f"{hand[2]}   Value: {format((hand[2].b >> 16), '#015b')[2:]} & {hand[2].MASK[-29:-16]}")
print(f"{hand[3]}   Value: {format((hand[3].b >> 16), '#015b')[2:]} & {hand[3].MASK[-29:-16]}")
print(f"{hand[4]}   Value: {format((hand[4].b >> 16), '#015b')[2:]} & {hand[4].MASK[-29:-16]}")
print(f"Combination:{format(GetEncoding(hand), '#015b')[2:]}")
print(f"Amount of unique valued cards: {bin(GetEncoding(hand)).count('1')}")

Hand: 8♢ K♡ 4♣ 2♠ 7♣
8♢   Value: 0000001000000 & AKQJT98765432
K♡   Value: 0100000000000 & AKQJT98765432
4♣   Value: 0000000000100 & AKQJT98765432
2♠   Value: 0000000000001 & AKQJT98765432
7♣   Value: 0000000100000 & AKQJT98765432
Combination:0100001100101
Amount of unique valued cards: 5


Now we can determine if a hand has a flush, straight flush or royal flush, and if it does have one; then we can determine what the strength of the hand is quantitatively.

In [170]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

if CheckFlush(hand):
    print("The hand contains a flush.")
    strength = FLUSH_RANKS[GetEncoding(hand)]
    print(f"The hand was ranked as the number {strength} strongest hand.")
else:
    print("The hand doesn't contain a flush.")

Hand: 5♠ 3♡ Q♡ 5♣ 10♣
The hand doesn't contain a flush.


#### Checking for Straights, or High Cards

The keys of the UNIQUE_FIVE_RANKS memo are populated bin the exact same way as for FLUSH_RANKS. Once they have been computed, they are then mapped to a number between 1 to 7462 which corresponds to the strength of the card combination that the key represents. Card combinations and strengths are enumerated from unique_five_lookup.txt.

In [122]:
UNIQUE_FIVE_RANKS = {
    VALUES[line[4]] | VALUES[line[5]] | 
    VALUES[line[6]] | VALUES[line[7]] | 
    VALUES[line[8]] : int(str(line)[11:]) 
    for line in open("lookup-tables/unique five lookup.txt", "r")}

Before a hand is used as a key for UNIQUE_FIVE_RANKS, the hand should be checked to see if it contains five unique valued cards. This is because not every hand is included in UNIQUE_FIVE_RANKS. Checking that the hand contains five unique valued cards can be done using a combination of the bitwise OR operation, the 32bit card integers, bitshifting and then counting how many bits are on. If the hand contains five cards with different values, then the result will have 5 bits switched on.

In [133]:
def CheckUniqueFive(hand):
    return (bin((hand[0].b | hand[1].b | hand[2].b | hand[3].b | hand[4].b) >> 16)).count("1") == 5

In [134]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

print(f"{hand[0]}   Value: {format((hand[0].b >> 16), '#015b')[2:]} & {hand[0].MASK[-29:-16]}")
print(f"{hand[1]}   Value: {format((hand[1].b >> 16), '#015b')[2:]} & {hand[1].MASK[-29:-16]}")
print(f"{hand[2]}   Value: {format((hand[2].b >> 16), '#015b')[2:]} & {hand[2].MASK[-29:-16]}")
print(f"{hand[3]}   Value: {format((hand[3].b >> 16), '#015b')[2:]} & {hand[3].MASK[-29:-16]}")
print(f"{hand[4]}   Value: {format((hand[4].b >> 16), '#015b')[2:]} & {hand[4].MASK[-29:-16]}")
print(f"Combination: {format(GetEncoding(hand), '#015b')[2:]}")
print(f"Amount of unique valued cards: {bin(GetEncoding(hand)).count('1')}")

Hand: 6♠ Q♢ 5♢ 2♢ 7♡
6♠   Value: 0000000010000 & AKQJT98765432
Q♢   Value: 0010000000000 & AKQJT98765432
5♢   Value: 0000000001000 & AKQJT98765432
2♢   Value: 0000000000001 & AKQJT98765432
7♡   Value: 0000000100000 & AKQJT98765432
Combination: 0010000111001
Amount of unique valued cards: 5


The encoding of a card combination to be used as a key for UNIQUE_FIVE_RANKS, is found the same way as it was for FLUSH_RANKS. The following code snippet ties everything together to determine whether a hand has a straight or a high card at most, and if it does have either; then what the strength is quantitatively.

In [135]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

if CheckUniqueFive(hand):
    print("The hand contains a straight or high card at most.")
    strength = UNIQUE_FIVE_RANKS[GetEncoding(hand)]
    print(f"The hand was ranked as the number {strength} strongest hand.")
else:
    print("The hand doesn't contain a straight or high card at most.")

Hand: 10♣ 3♡ 5♣ J♠ 2♢
The hand contains a straight or high card at most.
The hand was ranked as the number 7270 strongest hand.


#### Checking for Pairs, Two Pairs, Three of a Kinds, Full Houses or Four of a Kinds

The keys of the DUPE_RANKS memo are populated by multiplying together prime numbers associated with card values found in a card combination. The keys are mapped to a number between 1 to 7462 corresponding to the strength of card combination that the key represents. All combinations and strengths are taken from dupe_lookup.txt.

In [164]:
DUPE_RANKS = {
    PRIMES[line[4]] * PRIMES[line[5]] * 
    PRIMES[line[6]] * PRIMES[line[7]] * 
    PRIMES[line[8]] : int(str(line)[11:]) 
    for line in open("lookup-tables/dupe lookup.txt", "r")}

Before a hand is used as a key for DUPE_RANKS, it should be checked to see that it doesn't contain a flush, a straight  or high card at most. This is because not every hand is included in DUPE_RANKS. Checking the hand doesn't contains a flush or straight or high card at most, can be done through eliminative flow control.

In [137]:
def CheckDupe(hand):
    return not CheckFlush(hand) and not CheckUniqueFive(hand)

In [153]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

print(f"Hand has flush, straight flush or royal flush: {bool(CheckFlush(hand))}")
print(f"Hand has straight or high card at most: {bool(CheckUniqueFive(hand))}")
print(f"Hand has pair, two pair, three of a kind, full house or four of a kind: {bool(CheckDupe(hand))}")

Hand: 8♠ 7♡ 2♢ J♣ 3♢
Hand has flush, straight flush or royal flush: False
Hand has straight or high card at most: True
Hand has pair, two pair, three of a kind, full house or four of a kind: False


The encoding of a card combination to be used as a key for DUPE_RANKs, can be extracted using the bitwise AND operation, multiplication of primes, the 32bit card integers and a bitwise mask to extract the prime number associate for each card value in the card combination. The reason the encoding is different than for FLUSH_RANKS and UNIQUE_FIVE_RANKS, is to ensure information about which card is duplicated is contained in the encoding, which wasn't required for the other memos. 

In [154]:
def GetGodelNumber(hand):
    prime_mask = 255
    return (hand[0].b & prime_mask) * (hand[1].b & prime_mask) * (hand[2].b & prime_mask) * (hand[3].b & prime_mask) * (hand[4].b & prime_mask)

In [288]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

print(f"{hand[0]}   Prime: {hand[0].b & 255}")
print(f"{hand[1]}   Prime: {hand[1].b & 255}")
print(f"{hand[2]}   Prime: {hand[2].b & 255}")
print(f"{hand[3]}   Prime: {hand[3].b & 255}")
print(f"{hand[4]}   Prime: {hand[4].b & 255}")
print(f"Combination: {GetGodelNumber(hand)}")

Hand: 6♡ 4♠ 8♢ Q♡ K♡
6♡   Prime: 11
4♠   Prime: 5
8♢   Prime: 17
Q♡   Prime: 31
K♡   Prime: 37
Combination: 1072445


The following code snippet ties everything together to determine if a hand has a pair, two pair, three of a kind, full house or four of a kind, and if it does have any; what the strength of the hand is quantitatively.

In [182]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

if CheckDupe(hand):
    print("The hand contains a pair, two pair, three of a kind, full house or four of a kind.")
    strength = DUPE_RANKS[GetGodelNumber(hand)]
    print(f"The hand was ranked as the number {strength} strongest hand.")
else:
    print("The hand doesn't contain a pair, two pair, three of a kind, full house or four of a kind.")

Hand: 5♡ 3♠ 3♢ Q♠ 6♢
The hand contains a pair, two pair, three of a kind, full house or four of a kind.
The hand was ranked as the number 5876 strongest hand.


### Combination of Hand Evaluation Memos

With all of the memos and functions to encode card combinations and check hand conditions, it is now possible to get a hands strength as a quantity.

In [248]:
def GetStrength(hand):
    if CheckFlush(hand):
        return FLUSH_RANKS[GetEncoding(hand)]
    elif CheckUniqueFive(hand):
        return UNIQUE_FIVE_RANKS[GetEncoding(hand)]
    else:
        return DUPE_RANKS[GetGodelNumber(hand)]

In [289]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

strength = GetStrength(hand)
print(f"The hand was ranked as the number {strength} strongest hand.")

Hand: 3♡ K♠ A♡ 3♠ A♠
The hand was ranked as the number 2578 strongest hand.


### Categorising Hands

Hands aren't traditionally ranked numerically but instead they are ranked categorically. They are classified as either having a high card, a pair, two pairs, three of a kind, straight, flush, full house, four of a kind, straight flush or royal flush. To get this attribute, the following function can be used, which was devised using the lookup-tables as reference.

In [252]:
def GetCategory(strength):
    if 6186 <= strength:
        return "high card"
    elif 3326 <= strength and strength <= 6187:
        return "pair"
    elif 2468 <= strength and strength <= 3325:
        return "two pair"
    elif 1610 <= strength and strength <= 2467:
        return "three of a kind"
    elif 1600 <= strength and strength <= 1609:
        return "straight"
    elif 323 <= strength and strength <= 1599:
        return "flush"
    elif 167 <= strength and strength <= 322:
        return "full house"
    elif 11 <= strength and strength <= 166:
        return "four of a kind"
    elif 2 <= strength and strength <= 10:
        return "straigh flush"
    else:
        return "royal flush"

Now the strength of any hand can be found, both categorical and quantitative.

In [262]:
hand = Hand(deck.Shuffle()[:5])
print(f"Hand: {hand}")

strength = GetStrength(hand)
category = GetCategory(strength)
print(f"The hand was classed as {category}.")
print(f"The hand was ranked as the number {strength} strongest hand.")

Hand: 5♣ K♣ 3♣ 2♠ 8♢
The hand was classed as high card.
The hand was ranked as the number 6991 strongest hand.


## Swapping Cards in a Hand

In five card draw, players get the option to swap cards in their hand. The simplest way to do this is by creating a new hand object.

In [278]:
deck = Deck().Shuffle()
hand = Hand(deck[:5])
print(f"Hand before card swap: {hand}")

cards_to_keep = [card for card in hand if card.value_i == 12]
new_cards = deck[5:10-len(cards_to_keep)]

hand = Hand(cards_to_keep + new_cards)
print(f"Hand after card swap: {hand}")


Hand before card swap: 3♢ K♠ 3♠ 6♠ 6♡
Hand after card swap: 7♢ K♡ J♠ 9♢ 5♢
